In [57]:
import pyupbit as up
import datetime 
import numpy as np 
import pandas as pd 
import time 
import random 
import os 
import math  
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
from utils import *

class OLMAR:
    def __init__(self, epsilon, window_size):
        self.epsilon = epsilon
        self.window_size = window_size
        self.cumulative_wealth = 1.0

    def olmar(self, current_portfolio, predicted_returns, window_size):
        avg_x = np.sum([x*y for x,y in zip([1 for i in predicted_returns], predicted_returns)])
        avg_x *= 1.0 / float(len(predicted_returns))
        # numerator
        temp_right = self.epsilon - sum([x*y for x,y in zip(current_portfolio, predicted_returns)])
        # denominator
        temp_bottom1 = np.array(predicted_returns)
        temp_bottom2 = np.array([avg_x for i in predicted_returns])
        dist = np.sum(np.power((temp_bottom1 - temp_bottom2), 2))
        lambda_1 = max(0.0, temp_right/dist)
        # update portfolio
        np_current_portfolio = np.array(current_portfolio)
        new_portfolio = np_current_portfolio + lambda_1 * (temp_bottom1 - temp_bottom2)
        new_portfolio = simplex_projection(new_portfolio.tolist())
        return new_portfolio

    def predict(self, current_portfolio, current_return, previous_prices):
        # calculate cumulative wealth
        self.cumulative_wealth *= np.sum([x*y for x,y in zip(current_portfolio, current_return)])
        m = len(previous_prices[0]) # number of assets
        predicted_returns = []
        for i in range(m): # for each asset, calculate predicted return
            sum = 0
            for j in range(self.window_size):
                sum += previous_prices[j][i]
            predicted_return = (1/previous_prices[0][i]) * (1/self.window_size) * sum
            predicted_returns.append(predicted_return)

        new_portfolio = self.olmar(current_portfolio, predicted_returns, self.window_size)
        return new_portfolio

    def get_cumulative_wealth(self):
        return self.cumulative_wealth

olmar = OLMAR(10, 5) # epsilon = 10, w = 5
'''
current time step is t
since our window size is w = 5, we get the previous timesteps t,t-1,t-2,t-3,t-4
let us assume we have 4 assets (m = 4)
we repeat the below process for n (indefinite number of) timesteps
'''
current_portfolio = [0.25, 0.25, 0.25, 0.25]
previous_prices = [[100,120,130,110],  # t
                   [110,121,90,80], # t-1
                   [120,140,85,75], # t-2
                   [140,200,75,51], # t-3
                   [300,240,80,80]] # t-4
current_returns = [100/110, 120/121, 130/90, 110/80]


new_portfolio = olmar.predict(current_portfolio, current_returns, previous_prices)

print("portfolio at time t = {}".format(current_portfolio))
print("predicted portfolio at time t+1 = {}".format(new_portfolio))
print("cumulative wealth = {}".format(olmar.get_cumulative_wealth()))


portfolio at time t = [0.25, 0.25, 0.25, 0.25]
predicted portfolio at time t+1 = [1. 0. 0. 0.]
cumulative wealth = 1.180067722681359


# Get Price Information

From 2/26 to 3/31

Every 4 hours 

This is a typical example of a downtrending market. 

In [93]:
btc = up.get_ohlcv("KRW-BTC", interval="minute240",to="20210331")
eth = up.get_ohlcv("KRW-ETH", interval="minute240",to="20210331")
ada = up.get_ohlcv("KRW-ADA", interval="minute240",to="20210331") 
doge = up.get_ohlcv("KRW-DOGE", interval="minute240",to="20210331") 
xrp = up.get_ohlcv("KRW-XRP", interval="minute240",to="20210331")
dot = up.get_ohlcv("KRW-DOT", interval="minute240",to="20210331") 
bch = up.get_ohlcv("KRW-BCH", interval="minute240",to="20210331")
ltc = up.get_ohlcv("KRW-LTC",interval="minute240",to="20210331") 
eos = up.get_ohlcv("KRW-EOS",interval="minute240",to="20210331") 
link = up.get_ohlcv("KRW-LINK",interval="minute240",to="20210331") 

In [94]:
btc_close = btc['close'].values
eth_close = eth['close'].values 
ada_close = ada['close'].values
doge_close = doge['close'].values 
xrp_close = xrp['close'].values 
dot_close = dot['close'].values 
bch_close = bch['close'].values 
ltc_close = ltc['close'].values 
eos_close = eos['close'].values 
link_close = link['close'].values 

In [95]:
price_data = [btc_close, eth_close,
              ada_close, doge_close, 
              xrp_close, dot_close, 
              bch_close, ltc_close, 
              eos_close, link_close] 


In [96]:
olmar = OLMAR(10, 5)

In [97]:
m = 10 
w = 5
cnt = 1 
current_portfolio = [0.1 for i in range(m)] 
for i in range(w-1,len(btc_close)):
    current_returns = [] 
    for j in range(m): 
        ret = price_data[j][i]/price_data[j][i-1] 
        current_returns.append(ret) 
    previous_prices = [] 
    sub = 0 
    for j in range(w): 
        previous_price = [] 
        for k in range(m):
            previous_price.append(price_data[k][i-j])
        previous_prices.append(previous_price) 
    
    new_portfolio = olmar.predict(current_portfolio, current_returns, previous_prices) 
    print("Iteration = {}".format(cnt))
    print("portfolio at time t = {}".format(current_portfolio))
    print("predicted portfolio at time t+1 = {}".format(new_portfolio))
    print("cumulative wealth = {}".format(olmar.get_cumulative_wealth()))
    print()
    cnt += 1 



Iteration = 1
portfolio at time t = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
predicted portfolio at time t+1 = [2.051194869791978, -16.649631556093894, -156.89785753369756, -44.500837179793685, 38.56690054834551, 30.379402136134487, 44.00129053019919, 100.29047257142948, 3.8641618315117747, -0.10509621782829426]
cumulative wealth = 1.0340516090474428

Iteration = 2
portfolio at time t = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
predicted portfolio at time t+1 = [27.767049229089174, -11.721612236214686, -122.44481416683301, 38.13465407997272, 50.91519153668828, -52.164398531287354, 41.51647232287909, 46.37161761722348, 27.24542119802829, -44.619581049549204]
cumulative wealth = 1.059248290622836

Iteration = 3
portfolio at time t = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
predicted portfolio at time t+1 = [13.460031407421033, 22.854318893385127, -84.74862466753858, 19.900207371486395, 12.405448900272894, -6.813088845140012, 4.213453456775437, 11.626236564081

Iteration = 193
portfolio at time t = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
cumulative wealth = 1.2832750968943767

Iteration = 194
portfolio at time t = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
cumulative wealth = 1.2753794994446654

Iteration = 195
portfolio at time t = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
cumulative wealth = 1.2788173352819034

Iteration = 196
portfolio at time t = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
predicted portfolio at time t+1 = [-110.96073311104067, -157.59529946883308, 65.76152159469767, 2.285261571619402, 337.09520983349654, 12.19062362463816, -30.37393698611957, -87.82202366413726, -274.4107944336217, 244.8301710392452]
cumulative wealth = 1.2777487129352352

